In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np

In [2]:
df = pd.read_csv("../data/WHO-SIMPLE.csv")

In [3]:
socio_factors = [
    'NUTRITION_564',
    'LBW_PREVALENCE',
    'NUTRITION_HA_2',
    'NUTRITION_WA_2',
    'NUTRITION_WH2',
    'NUTRITION_WH_2',
    'MDG_0000000026',
    'WHS_PBR', 
    'WSH_SANITATION_SAFELY_MANAGED', 
    'M_Est_smk_curr',
    'M_Est_smk_daily',
    'TOBACCO_0000000192',
    'GHED_CHEGDP_SHA2011', 
    'GDP',
    'MDG_0000000003', 
    'MDG_0000000025'
]

imr = 'MDG_0000000001'

In [4]:
relevant_factors = socio_factors.copy()
relevant_factors.insert(0, imr)

In [5]:
socio_df = df[relevant_factors].copy()

In [6]:
socio_df = socio_df.dropna(thresh=2000, axis=1)

In [7]:
socio_df.describe()

,MDG_0000000001,LBW_PREVALENCE,MDG_0000000026,GHED_CHEGDP_SHA2011,GDP,MDG_0000000003
count,11289.000000,2320.000000,3276.000000,3560.000000,8.911000e+03,2176.000000
mean,54.199007,10.087026,207.671245,6.300938,1.914292e+11,43.188787
std,48.087793,4.923792,302.623492,3.013298,9.744936e+11,40.854141
min,1.493180,2.400000,2.000000,1.030000,8.824448e+06,0.000000
25%,15.539030,6.200000,16.000000,4.240000,1.666522e+09,12.875000
50%,38.974950,8.800000,65.000000,5.785000,9.059340e+09,28.000000
75%,82.065320,12.800000,290.250000,7.960000,5.758224e+10,62.400000
max,279.657810,36.200000,2480.000000,50.180000,2.143323e+13,229.000000


In [8]:
len(socio_df)

11289

In [9]:
socio_df = socio_df.dropna(how='any')

In [10]:
len(socio_df)

1622

## Center and scale data for each columns

In [11]:
all_columns = socio_df.columns

In [12]:
# Create the Scalar object
scaler = preprocessing.StandardScaler()

In [13]:
# Fit data on the Scaler object
scaled_arr = scaler.fit_transform(socio_df[all_columns])
scaled_df = pd.DataFrame(scaled_arr, columns=all_columns)

In [14]:
scaled_df.head()

,MDG_0000000001,LBW_PREVALENCE,MDG_0000000026,GHED_CHEGDP_SHA2011,GDP,MDG_0000000003
0,4.628108,1.964025,3.003983,-0.834606,-0.297514,2.176267
1,4.260804,1.874647,2.374774,-1.201911,-0.292163,2.934001
2,3.820671,1.785268,1.877790,-1.478378,-0.280614,3.320817
3,3.106338,1.651201,1.312413,-1.292750,-0.249529,3.914287
4,2.878960,1.606511,1.148272,-1.087375,-0.260519,3.991120


## Random Forest Implementation

In [15]:
# Separate explanatory variables and response variable
explanatory_vars = all_columns.copy()
explanatory_vars = list(explanatory_vars)
explanatory_vars.remove(imr)

# Split data into train and test sets
x = scaled_df[explanatory_vars]
y = scaled_df[imr]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [16]:
len(x_train)

1086

In [17]:
len(x_test)

536

In [18]:
rf = RandomForestRegressor()

In [19]:
rf.fit(x_train, y_train)

C:\Users\Daniel\AppData\Local\conda\conda\envs\Research\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [20]:
rf.score(x_test, y_test)

0.941990384894389

In [21]:
feature_importances = rf.feature_importances_

In [22]:
for i, feature in enumerate(explanatory_vars):
    print("Feature:", feature, "Importance:", feature_importances[i])

Feature: LBW_PREVALENCE Importance: 0.03744586496921695
Feature: MDG_0000000026 Importance: 0.8730716878467824
Feature: GHED_CHEGDP_SHA2011 Importance: 0.026680609760405875
Feature: GDP Importance: 0.03125705458984509
Feature: MDG_0000000003 Importance: 0.03154478283374968
